In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySpark Example").getOrCreate()

24/12/03 10:27:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## RDD 객체

In [3]:
rdd = spark.sparkContext.parallelize([1,2,3,4,5])
rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [4]:
squared_rdd = rdd.map(lambda x:x*x)
squared_rdd

PythonRDD[1] at RDD at PythonRDD.scala:53

In [5]:
rdd.take(5)

[1, 2, 3, 4, 5]

In [6]:
squared_rdd.take(5)

[1, 4, 9, 16, 25]

In [7]:
squared_rdd.collect()

[1, 4, 9, 16, 25]

##  http://15.168.145.74:4040/jobs/ 에서 작업 현황 확인

## 데이터프레임 객체

In [8]:
data = [('Alice',1), ('Bob',2), ('Charlie',3)]
df = spark.createDataFrame(data, ['Name','Value'])
df

DataFrame[Name: string, Value: bigint]

In [9]:
df.show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|    Bob|    2|
|Charlie|    3|
+-------+-----+



## RDBMS - 테이블

In [10]:
df.createOrReplaceTempView('people')

In [15]:
select_sql = "SELECT * FROM people WHERE Value > 2"

In [16]:
result_sql = spark.sql(select_sql)
result_sql.show()

+-------+-----+
|   Name|Value|
+-------+-----+
|Charlie|    3|
+-------+-----+



In [18]:
data2 = [('John',1, 'Sales'), ('Bob',2, 'Marketing'), ('Charlie',3, 'Finance'),
        ('Smith', 4, 'R&D'), ('Kate',5, 'Operations'), ('Jack',6,'Legal'),
        ('Tom', 7, 'HR'), ('Kim',8,'IT')]
df2 = spark.createDataFrame(data2, ['Name','Id','Department'])
df2

DataFrame[Name: string, Id: bigint, Department: string]

In [19]:
df2.show()

+-------+---+----------+
|   Name| Id|Department|
+-------+---+----------+
|   John|  1|     Sales|
|    Bob|  2| Marketing|
|Charlie|  3|   Finance|
|  Smith|  4|       R&D|
|   Kate|  5|Operations|
|   Jack|  6|     Legal|
|    Tom|  7|        HR|
|    Kim|  8|        IT|
+-------+---+----------+



## MLlib

In [22]:
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 91.4 MB/s eta 0:00:00


In [23]:
import numpy as np

In [24]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [26]:
data3 = [('Alice',1), ('Bob',2), ('Charlie',3)]
df3 = spark.createDataFrame(data, ['Name','age'])

df3_filtered = df3.filter(df3.age > 2)
df3_filtered.show()

+-------+---+
|   Name|age|
+-------+---+
|Charlie|  3|
+-------+---+



In [27]:
assembler = VectorAssembler(inputCols=['age'], outputCol='features')
vector_df3 = assembler.transform(df3)

In [28]:
lr = LinearRegression(featuresCol='features', labelCol='age')
model = lr.fit(vector_df3)

24/12/03 11:30:26 WARN Instrumentation: [71a2e23f] regParam is zero, which might cause numerical instability and overfitting.
24/12/03 11:30:27 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/12/03 11:30:27 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/12/03 11:30:27 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/12/03 11:30:27 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [29]:
pred = model.transform(vector_df3)
pred.show()

+-------+---+--------+------------------+
|   Name|age|features|        prediction|
+-------+---+--------+------------------+
|  Alice|  1|   [1.0]|0.9999999999999986|
|    Bob|  2|   [2.0]|1.9999999999999998|
|Charlie|  3|   [3.0]| 3.000000000000001|
+-------+---+--------+------------------+



In [30]:
spark.stop()